In [ ]:
# --- CELL 1: SETUP ---
from google.colab import drive
import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow
from IPython.display import Image, display
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import glob

In [ ]:
# Mount Google Drive to save our results persistently
drive.mount('/content/drive')

# Define the main project folder in your Google Drive
project_folder = '/content/drive/MyDrive/YOLO_Pothole_Results'
os.makedirs(project_folder, exist_ok=True)

In [ ]:
!pip install ultralytics roboflow -q


In [ ]:
# --- DOWNLOAD DATASET ---
# IMPORTANT: Replace with your own Roboflow API Key
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")

# Download the specified dataset version from your Roboflow project
project = rf.workspace("aegis").project("pothole-detection-i00zy")
dataset = project.version(10).download("yolov8")

# Store the dataset path for later use
dataset_path = dataset.location

In [ ]:
# --- VISUALIZE DATASET ---
# Get a list of 4 random training images
image_files = glob.glob(os.path.join(dataset_path, 'train/images', '*.jpg'))[:4]

# Plot the images
plt.figure(figsize=(14, 8))
for i, img_path in enumerate(image_files):
    plt.subplot(2, 2, i + 1)
    img = PILImage.open(img_path)
    plt.imshow(img)
    plt.title(f'Sample Image {i+1}')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# --- TRAIN THE MODEL ---
# Define the model checkpoint to load. Use a pre-trained model for the first run.
# After the first run, you can change this to the 'last.pt' file in your results folder to resume.
model_checkpoint_path = '/content/drive/MyDrive/YOLO_Pothole_Results/pothole_detector_yolov8m_final/weights/last.pt'

if os.path.exists(model_checkpoint_path):
    print("Resuming training from last checkpoint...")
    model = YOLO(model_checkpoint_path)
else:
    print("Starting new training from pretrained yolov8m.pt...")
    model = YOLO('yolov8m.pt')

# Start or resume training
results = model.train(
    data=os.path.join(dataset_path, 'data.yaml'),
    epochs=200,
    imgsz=640,
    project=project_folder,       # Saves results to your Google Drive
    name='pothole_detector_yolov8m_final',
    exist_ok=True,              # Allows resuming in the same folder
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    bgr=0.0,
    mosaic=1.0,
    mixup=0.0,
    cutmix=0.0,
    copy_paste=0.0,
    device=0                    # Use GPU (0)
)

In [ ]:
# --- EVALUATE PERFORMANCE ---
# Define the path to the training results folder
results_path = os.path.join(project_folder, 'pothole_detector_yolov8m_final')

# Display the confusion matrix
print("Confusion Matrix:")
display(Image(filename=os.path.join(results_path, 'confusion_matrix.png'), width=800))

# Display the training and validation loss curves
print("\nTraining/Validation Results:")
display(Image(filename=os.path.join(results_path, 'results.png'), width=1000))

# Display the Precision-Recall curve
print("\nPrecision-Recall Curve:")
display(Image(filename=os.path.join(results_path, 'PR_curve.png'), width=800))

In [ ]:
# --- VALIDATE BEST MODEL ---
# Load the best performing model from the training run
best_model_path = os.path.join(results_path, 'weights/best.pt')
model = YOLO(best_model_path)

# Run validation
metrics = model.val()
print("\n--- Final Model Metrics ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"Precision: {metrics.box.p[0]:.4f}")
print(f"Recall: {metrics.box.r[0]:.4f}")

In [ ]:
# --- RUN INFERENCE ---
# Get a list of a few test images
test_image_files = glob.glob(os.path.join(dataset_path, 'test/images', '*.jpg'))[:3]

# Run prediction on the test images
results = model.predict(test_image_files, save=True)

# Display the result images with bounding boxes
print("\n--- Inference Results ---")
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB
    display(im)